# Generating SQL for Microsoft SQL Server using OpenAI via Vanna.AI and Vanna Hosted Vector DB
This notebook runs through the process of using the `vanna` Python package to generate SQL using AI (RAG + LLMs) including connecting to a database and training.

To use this demo, ensure you have access to a SQL Server/Azure SQL Database with the adventure works sample data installed. You will use the username, password, server location, and database name when constructing the connect string. Also included in this directory is the **odbcDriverInstallUbuntu.txt** script that you can use to install the ODBC Driver for SQL on Linux (Ubuntu) in a codespace.

## Setup

In [ ]:
%pip install vanna pyodbc

In [ ]:
import vanna
import pyodbc
from vanna.remote import VannaDefault

In [ ]:
api_key = '1234567890' # Your API key from https://vanna.ai/account/profile 

vanna_model_name = 'YOUR_VANNA_MODEL_NAME' # Your model name from https://vanna.ai/account/profile 
vn = VannaDefault(model=vanna_model_name, api_key=api_key)


## Query the Azure SQL Database
Choose one of the following authentication methods

### Using SQL Authentication:

In [ ]:
vn.connect_to_mssql(odbc_conn_str='DRIVER={ODBC Driver 18 for SQL Server};Server=tcp:<SERVER_NAME>,1433;Database=<DATABASE_NAME>;Uid=<USER_NAME>;Pwd=<SQL_PASSWORD>;Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;') # You can use the ODBC connection string here

### Using Entra ID Service Principle Authentication

In [ ]:
vn.connect_to_mssql(odbc_conn_str='DRIVER={ODBC Driver 18 for SQL Server};Server=tcp:<SERVER_NAME>,1433;Database=<DATABASE_NAME>;Uid=<USER_NAME>;Pwd=<SQL_PASSWORD>;Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;Authentication=ActiveDirectoryServicePrincipal') # You can use the ODBC connection string here

## Training
You only need to train once. Do not train again unless you want to add more training data.

In [ ]:

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
# vn.train(plan=plan)



In [ ]:
vn.train(documentation="When querying any table, use the fully qualified table name including schema")
# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [ ]:

# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")


In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

In [ ]:
# You can remove training data if there's obsolete/incorrect information. 
vn.remove_training_data(id='what are the top products sold and their colors?')


## Asking the AI
Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.

In [ ]:
vn.ask(question='what are the top products sold and their colors?')

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

## Next Steps
Using Vanna via Jupyter notebooks is great for getting started but check out additional customizable interfaces like the 
- [Streamlit app](https://github.com/vanna-ai/vanna-streamlit)
- [Flask app](https://github.com/vanna-ai/vanna-flask)
- [Slackbot](https://github.com/vanna-ai/vanna-slack)
